In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
import json

In [3]:
f = open("C:\\Users\\91840\\Downloads\\loan_data.json")

data = json.load(f)

In [4]:
data[:2]

[{'Application_ID': 'LP001002',
  'Gender': 'Male',
  'Married': 'No',
  'Dependents': '0',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'medium',
  'Application_Status': 'Y'},
 {'Application_ID': 'LP001003',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '1',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Rural',
  'Income': 'medium',
  'Application_Status': 'N'}]

In [5]:
df=pd.DataFrame.from_dict(data, orient='columns')

In [6]:
df.head()

,Application_ID,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,Application_Status
0,LP001002,Male,No,0,Graduate,No,1,Urban,medium,Y
1,LP001003,Male,Yes,1,Graduate,No,1,Rural,medium,N
2,LP001005,Male,Yes,0,Graduate,Yes,1,Urban,low,Y
3,LP001006,Male,Yes,0,Not Graduate,No,1,Urban,low,Y
4,LP001008,Male,No,0,Graduate,No,1,Urban,medium,Y


# Q.no1

1. Find % of total applicants for each unique value of dependents.

In [7]:
unique_values=df.Dependents.value_counts().index
total=df.Dependents.count()

idx=0

for i in df.Dependents.value_counts():
  print(unique_values[idx],":",round(i*100/total,2),'%')
  idx=idx+1

0 : 57.53 %
2 : 17.22 %
1 : 16.63 %
3+ : 8.61 %


# Q.no 2


2. Find the %of applications approved for self-employed applicants.

In [9]:
total_applicants= df.Application_ID.count()
print("approved %: ",round(df.Application_Status[df['Application_Status']=='Y'].count()*100/total_applicants,2),"%")

approved %:  67.91 %


3. What is the % of rejections for married male applicants?

In [14]:
print("Rejection %: ",round(df.loc[(df.Gender=="Male")&(df.Married=="Yes")&(df.Application_Status=="N")]['Application_ID'].count()*100/total_applicants,2),"%")

Rejection %:  17.03 %


4. Which property area has the maximum approval ratio

In [15]:
df.groupby('Application_Status')['Property_Area'].value_counts()

Application_Status  Property_Area
N                   Urban             61
                    Rural             59
                    Semiurban         44
Y                   Semiurban        153
                    Urban            104
                    Rural             90
Name: Property_Area, dtype: int64

In [16]:
def approval_rate(name):
  # print(df.loc[(df.Property_Area==name)].Application_Status.value_counts())
  y=df.loc[(df.Property_Area==name)].Application_Status.value_counts()[0]
  n=df.loc[(df.Property_Area==name)].Application_Status.value_counts()[1]
  print("-------------")
  print("approval rate of ",name,": ",y*100/(y+n),"%")
  print("-------------")

In [17]:
df.Property_Area.value_counts()

Semiurban    197
Urban        165
Rural        149
Name: Property_Area, dtype: int64

In [18]:
df.Property_Area.value_counts().index[0]

'Semiurban'

In [19]:
pd.Series(df.Property_Area.value_counts().index).apply(lambda x:approval_rate(x))

-------------
approval rate of  Semiurban :  77.66497461928934 %
-------------
-------------
approval rate of  Urban :  63.03030303030303 %
-------------
-------------
approval rate of  Rural :  60.40268456375839 %
-------------


0    None
1    None
2    None
dtype: object

highest approval rate is of semiurban 77.664%

5. Find average number of dependents per income group.

In [22]:
def convert_string(a):
  if(a=="0"):
    return 0
  elif(a=="1"):
    return 1
  elif(a=="2"):
    return 2
  
  return 3
df['dependent']=df['Dependents'].apply(lambda x:convert_string(x))

df.groupby('Income')['dependent'].sum()/df.groupby('Income')['Application_ID'].count()

Income
high      0.888889
low       0.641026
medium    0.922280
dtype: float64

# model

In [23]:
df.head()

,Application_ID,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,Application_Status,dependent
0,LP001002,Male,No,0,Graduate,No,1,Urban,medium,Y,0
1,LP001003,Male,Yes,1,Graduate,No,1,Rural,medium,N,1
2,LP001005,Male,Yes,0,Graduate,Yes,1,Urban,low,Y,0
3,LP001006,Male,Yes,0,Not Graduate,No,1,Urban,low,Y,0
4,LP001008,Male,No,0,Graduate,No,1,Urban,medium,Y,0


In [24]:
df.isnull().sum()

Application_ID        0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
Credit_History        0
Property_Area         0
Income                0
Application_Status    0
dependent             0
dtype: int64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Application_ID      511 non-null    object
 1   Gender              511 non-null    object
 2   Married             511 non-null    object
 3   Dependents          511 non-null    object
 4   Education           511 non-null    object
 5   Self_Employed       511 non-null    object
 6   Credit_History      511 non-null    int64 
 7   Property_Area       511 non-null    object
 8   Income              511 non-null    object
 9   Application_Status  511 non-null    object
 10  dependent           511 non-null    int64 
dtypes: int64(2), object(9)
memory usage: 44.0+ KB


In [26]:
data=df.copy()
data.head()

,Application_ID,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,Application_Status,dependent
0,LP001002,Male,No,0,Graduate,No,1,Urban,medium,Y,0
1,LP001003,Male,Yes,1,Graduate,No,1,Rural,medium,N,1
2,LP001005,Male,Yes,0,Graduate,Yes,1,Urban,low,Y,0
3,LP001006,Male,Yes,0,Not Graduate,No,1,Urban,low,Y,0
4,LP001008,Male,No,0,Graduate,No,1,Urban,medium,Y,0


In [27]:
data=data.drop(columns=['Application_ID','Dependents'])

In [28]:
data.Gender.value_counts()

Male      420
Female     91
Name: Gender, dtype: int64

In [29]:
data['Gender']=data['Gender'].apply(lambda x: 1 if x=="Female" else 0)

In [30]:
data['Married']=data['Married'].apply(lambda x: 1 if x=="Yes" else 0) 
data['Education']=data['Education'].apply(lambda x: 1 if x=="Graduate" else 0) 
data['Application_Status']=data['Application_Status'].apply(lambda x: 1 if x=="Y" else 0) 
data['Self_Employed']=data['Self_Employed'].apply(lambda x: 1 if x=="Yes" else 0)

In [31]:
data=data.join(pd.get_dummies(df.Property_Area,prefix='property_area'))

In [32]:
data=data.join(pd.get_dummies(df.Income,prefix='income'))

In [33]:
data.head()

,Gender,Married,Education,Self_Employed,Credit_History,Property_Area,Income,Application_Status,dependent,property_area_Rural,property_area_Semiurban,property_area_Urban,income_high,income_low,income_medium
0,0,0,1,0,1,Urban,medium,1,0,0,0,1,0,0,1
1,0,1,1,0,1,Rural,medium,0,1,1,0,0,0,0,1
2,0,1,1,1,1,Urban,low,1,0,0,0,1,0,1,0
3,0,1,0,0,1,Urban,low,1,0,0,0,1,0,1,0
4,0,0,1,0,1,Urban,medium,1,0,0,0,1,0,0,1


In [34]:
data=data.drop(columns=['Property_Area','Income'])

In [35]:

from sklearn.model_selection import train_test_split
X= data.drop(columns=['Application_Status'])
Y= data['Application_Status']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42, stratify=data['Application_Status'])

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [37]:
tree= DecisionTreeClassifier(random_state=24) 
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
lr = LogisticRegression(random_state=24)


models= [tree, forest, knn, svm,lr]
acc=[]
max_acc=0

for model in models:
    model.fit(X_train, y_train) 
    y_pred= model.predict(X_test) 
    accuracy= accuracy_score(y_test, y_pred) 
     
    print(f"The accuracy of model {type(model).__name__} is {accuracy*100:.2f}")

    if(accuracy>max_acc):
      model_name=type(model).__name__
      max_acc=accuracy
      best_model=model

    
    print("\n")

print(f"Highest accuracy has been obtained using model {model_name}, accuracy:{max_acc*100:.2f}%.")

The accuracy of model DecisionTreeClassifier is 76.62


The accuracy of model RandomForestClassifier is 76.62


The accuracy of model KNeighborsClassifier is 78.57


The accuracy of model SVC is 84.42


The accuracy of model LogisticRegression is 83.12


Highest accuracy has been obtained using model SVC, accuracy:84.42%.


In [38]:
def take_inputs():
  x_pred=[]
  print("Gender: if Male enter 0, if Female enter 1")
  gender=int(input())
  x_pred.append(gender)

  print("Married: Yes: 1 No:0")
  married=int(input())
  x_pred.append(married)

  print("Education: graduated: 1 Not graduated:0")
  edu=int(input())
  x_pred.append(edu)

  print("Selfemployed? Yes 1, no 0")
  self_employed=int(input())
  x_pred.append(self_employed)

  print("Credit History? Yes 1, No 0")
  credit_history=int(input())
  x_pred.append(credit_history)

  print("dependent?, if 0: enter 0, if 1: enter 1, if 2: enter 2, if 3+:enter 3")
  dependent=int(input())
  x_pred.append(dependent)

  print("property area type: 1: rural, 2: semi urban, 3: urban")
  area=int(input())

  if(area ==1):
    x_pred=x_pred+[1,0,0]

  elif(area==2):
    x_pred=x_pred+[0,1,0]

  elif(area==3):
    x_pred=x_pred+[0,0,1]
  
  print("income type: 1: high, 2:low, 3: medium")
  income=int(input())  
  if(income==1):
    x_pred=x_pred+[1,0,0]

  elif(income==2):
    x_pred=x_pred+[0,1,0]
  elif(income==3):
    x_pred=x_pred+[0,0,1]

  return x_pred

In [39]:
def loan_application_status():
  x_pred=take_inputs()
  y_pred=best_model.predict([x_pred])[0]
  print("******************************************")
  if(y_pred==0):
    print("loan application will be rejected")
  else:
    print("loan application will be approved")
  
  print("******************************************")

In [40]:
loan_application_status()

Gender: if Male enter 0, if Female enter 1
1
Married: Yes: 1 No:0
1
Education: graduated: 1 Not graduated:0
1
Selfemployed? Yes 1, no 0
1
Credit History? Yes 1, No 0
1
dependent?, if 0: enter 0, if 1: enter 1, if 2: enter 2, if 3+:enter 3
1
property area type: 1: rural, 2: semi urban, 3: urban
1
income type: 1: high, 2:low, 3: medium
1
******************************************
loan application will be approved
******************************************


C:\Users\91840\anaconda3\envs\open_cv\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"
